# CSE547 - Colab 1
## Wordcount in Spark

Adapted From Stanford CS246

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/cli/main.py", line 9, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/cli/autocompletion.py", line 12, in <module>
    from pip._internal.metadata import get_default_environment
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/metadata/__init__.py", line 3, in <module>
    from .base import BaseDistribution, BaseEnvironment, FilesystemWheel, MemoryWheel, Wheel
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/metadata/base.py", line 21, in <module>
    from pip._vendor.packaging.requirements import Requirement
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/packaging/requirements.py", line 10, in <module>
    from pip._vendor.pyparsing import (  # noqa
  File "/usr/local/lib/pytho

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='1SE6k_0YukzGd5wK-E4i6mG83nydlfvSa'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('pg100.txt')

If you executed the cells above, you should be able to see the file *pg100.txt* under the "Files" tab on the left panel.

### Your task

If you run successfully the setup stage, you are ready to work on the *pg100.txt* file which contains a copy of the complete works of Shakespeare.

Write a Spark application which outputs the number of words that start with each letter. This means that for every letter we want to count the total number of (non-unique) words that start with a specific letter. In your implementation **ignore the letter case**, i.e., consider all words as lower case. Also, you can ignore all the words **starting** with a non-alphabetic character.

For this task we ask you to the [**RDD MapReduce API**](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.html) from spark (map, reduceByKey, flatMap, etc.) instead of **DataFrame API**. 

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd
import re

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [ ]:
f = sc.textFile("/content/pg100.txt")
tokens= f.flatMap(lambda line : line.strip().lower().split(" "))
words = tokens.map(lambda w: re.sub(r'[^a-z]', 'z', w)).filter(lambda w: len(w) > 0)
word_counts= words.map(lambda word: (list(word)[0],1)).reduceByKey(lambda a,b:a+b)

In [ ]:
#word_counts= tokens.map(lambda word: (list(word)[0],1)).reduceByKey(lambda a,b:a+b)

In [ ]:
sorted_word_counts = word_counts.sortBy(lambda key : -key[1]).collect()

In [ ]:
sorted_word_counts

[('t', 123602),
 ('a', 84836),
 ('s', 65705),
 ('i', 62167),
 ('h', 60563),
 ('w', 59597),
 ('m', 55676),
 ('b', 45455),
 ('o', 43494),
 ('f', 36814),
 ('c', 34567),
 ('d', 29713),
 ('l', 29569),
 ('p', 27759),
 ('n', 26759),
 ('y', 25855),
 ('g', 20782),
 ('e', 18697),
 ('r', 14265),
 ('k', 9418),
 ('u', 9170),
 ('z', 8140),
 ('v', 5728),
 ('j', 3339),
 ('q', 2377),
 ('x', 14)]

In [ ]:
res = 0

for i in sorted_word_counts:
  if i[0] and i[0][0] == "t":
    res += i[1]

res

123602

In [ ]:
res = 0

for i in sorted_word_counts:
  if i[0] and i[0][0] == "i":
    res += i[1]

res

62167

In [ ]:
res = 0

for i in sorted_word_counts:
  if i[0] and i[0][0] == "m":
    res += i[1]

res

55676

In [ ]:
Titles.registerTempTable("Titles")

query = """
SELECT titleType, count(*) as numTitles
FROM Titles
WHERE 
"""

title_type_counts = spark.sql(query)
title_type_counts.show()

Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!